In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/DL/anime-dataset-2023.csv')
df.head()

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,34784,Kuroko no Basket: Oshaberi Shiyou ka,UNKNOWN,黒子のバスケ お喋りしようか,7.18,"Comedy, Sports",Special included on Kuroko no Basket Fan Disc ...,Special,1.0,"Jan 27, 2017",...,Production I.G,Manga,15 min,PG-13 - Teens 13 or older,3125.0,5019,8,5448.0,15940,https://cdn.myanimelist.net/images/anime/9/839...
1,3464,Kemeko Deluxe!,UNKNOWN,ケメコデラックス！,6.55,"Action, Comedy, Romance, Sci-Fi, Ecchi","When Sanpeita Kobayashi was a child, his lovel...",TV,12.0,"Oct 5, 2008 to Dec 21, 2008",...,Hal Film Maker,Manga,24 min per ep,PG-13 - Teens 13 or older,6054.0,3232,55,16947.0,41132,https://cdn.myanimelist.net/images/anime/3/755...
2,54253,"Sekai, Nishihara Shoukai no Sekai! Part 2 Gyak...",UNKNOWN,世界、西原商会の世界！ Part 2 逆featuring CRAZY KEN BAND,UNKNOWN,UNKNOWN,"Music video for the song Sekai, Nishihara Shou...",Music,1.0,"May 25, 2022",...,"Sunrise, studio NAGURI",Original,6 min,G - All Ages,UNKNOWN,20918,0,UNKNOWN,85,https://cdn.myanimelist.net/images/anime/1247/...
3,5997,Sabu to Ichi Torimono Hikae,Sabu & Ichi's Arrest Warrant,佐武と市捕物控,6.69,"Action, Adventure, Drama, Slice of Life","The series follows the adventures of Sabu, a y...",TV,52.0,"Oct 3, 1968 to Sep 24, 1969",...,Toei Animation,Manga,25 min per ep,R - 17+ (violence & profanity),5396.0,10178,10,354.0,2342,https://cdn.myanimelist.net/images/anime/9/840...
4,6425,Weiß Survive,UNKNOWN,ヴァイス・サヴァイヴ,6.03,"Comedy, Ecchi",Takeshi was just an ordinary guy until a perfe...,TV,16.0,"Jun 6, 2009 to Sep 26, 2009",...,Studio Hibari,Card game,2 min per ep,PG-13 - Teens 13 or older,8714.0,4483,7,8355.0,20867,https://cdn.myanimelist.net/images/anime/10/17...


In [ ]:
selected_columns = ['anime_id', 'Name', 'Genres', 'Image URL']
df = df[selected_columns]

df.head()

,anime_id,Name,Genres,Image URL
0,34784,Kuroko no Basket: Oshaberi Shiyou ka,"Comedy, Sports",https://cdn.myanimelist.net/images/anime/9/839...
1,3464,Kemeko Deluxe!,"Action, Comedy, Romance, Sci-Fi, Ecchi",https://cdn.myanimelist.net/images/anime/3/755...
2,54253,"Sekai, Nishihara Shoukai no Sekai! Part 2 Gyak...",UNKNOWN,https://cdn.myanimelist.net/images/anime/1247/...
3,5997,Sabu to Ichi Torimono Hikae,"Action, Adventure, Drama, Slice of Life",https://cdn.myanimelist.net/images/anime/9/840...
4,6425,Weiß Survive,"Comedy, Ecchi",https://cdn.myanimelist.net/images/anime/10/17...


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

df['Genres'] = df['Genres'].apply(lambda x: x.split(', '))
mlb = MultiLabelBinarizer()

one_hot_genres = mlb.fit_transform(df['Genres'])


anime = pd.DataFrame(one_hot_genres, columns=mlb.classes_)


anime = pd.concat([df, anime], axis=1)

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import numpy as np

def preprocess_image(url, target_size=(224, 224)):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            img = img.resize(target_size)
            img_array = np.array(img) / 255.0
            return img_array
        else:
            return np.zeros((target_size[0], target_size[1], 3))
    except Exception as e:
        print(f"Error processing image from {url}: {e}")
        return np.zeros((target_size[0], target_size[1], 3))

In [ ]:
anime['processed_images'] = anime['Image URL'].apply(lambda url: preprocess_image(url))

In [ ]:
desired_shape = (224, 224, 3)
anime = anime[anime['processed_images'].apply(lambda img: img.shape == desired_shape)]

In [ ]:
one_hot_genres = mlb.fit_transform(anime['Genres'])

In [ ]:
X = np.array(anime['processed_images'].tolist())
y = one_hot_genres

In [ ]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow.keras.backend as K
import tensorflow as tf

def hammingloss(y_true, y_pred):
    # threshold the predictions at 0.7
    y_pred = K.cast(y_pred > 0.7, 'float32')
    return K.mean(K.not_equal(y_true, y_pred), axis=-1)

In [ ]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.8/584.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.4 MB/s eta 0:00:00


In [ ]:
import autokeras as ak

Using TensorFlow backend


In [ ]:
from sklearn.metrics import hamming_loss

class HammingLossCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        y_pred = self.model.predict(self.validation_data[0])
        y_pred_thresholded = (y_pred > 0.7).astype(int)
        hamming_loss_value = hamming_loss(self.validation_data[1], y_pred_thresholded)
        logs['val_hamming_loss'] = hamming_loss_value
        print(f' - val_hamming_loss: {hamming_loss_value:.4f}')

In [ ]:
clf = ak.ImageClassifier(max_trials=3)

Reloading Tuner from ./image_classifier/tuner0.json


In [ ]:
clf.fit(train_images, train_labels, epochs=10)

Trial 3 Complete [01h 28m 49s]
val_loss: 4.955391883850098

Best val_loss So Far: 4.955391883850098
Total elapsed time: 04h 20m 35s
Epoch 1/10
25/25 [==============================] - 651s 22s/step - loss: 5.3703 - accuracy: 0.0672
Epoch 2/10
25/25 [==============================] - 558s 22s/step - loss: 5.1919 - accuracy: 0.1318
Epoch 3/10
25/25 [==============================] - 549s 22s/step - loss: 5.0183 - accuracy: 0.2205
Epoch 4/10
25/25 [==============================] - 545s 22s/step - loss: 4.8345 - accuracy: 0.3207
Epoch 5/10
25/25 [==============================] - 550s 22s/step - loss: 4.6386 - accuracy: 0.3511
Epoch 6/10
25/25 [==============================] - 564s 23s/step - loss: 4.4414 - accuracy: 0.4525
Epoch 7/10
25/25 [==============================] - 564s 23s/step - loss: 4.2426 - accuracy: 0.5120
Epoch 8/10
25/25 [==============================] - 525s 21s/step - loss: 4.0568 - accuracy: 0.5146
Epoch 9/10
25/25 [==============================] - 521s 21s/step - 

In [ ]:
accuracy = clf.evaluate(test_images, test_labels)
print("Accuracy: ", accuracy)

model = clf.export_model()
model.save('best_model.h5')

7/7 [==============================] - 61s 5s/step - loss: 5.1449 - accuracy: 0.1919
Accuracy:  [5.144929885864258, 0.19191919267177582]


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
hamming_loss_callback = HammingLossCallback()

history = clf.fit(
    train_images,
    train_labels,
    epochs=3,
    validation_split=0.2,
    callbacks=[hamming_loss_callback]
)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['val_accuracy']
loss = history.history['val_loss']
hamming_loss_values = [log['val_hamming_loss'] for log in hamming_loss_callback.logs]

epochs = range(1, len(acc) + 1)

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.plot(epochs, acc, 'b', label='Validation Accuracy')
plt.title('Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(epochs, loss, 'r', label='Validation Loss')
plt.title('Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 3)
plt.plot(epochs, hamming_loss_values, 'g', label='Validation Hamming Loss')
plt.title('Validation Hamming Loss')
plt.xlabel('Epoch')
plt.ylabel('Hamming Loss')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
from tensorflow.keras.metrics import Precision, Recall
import numpy as np
from sklearn.metrics import hamming_loss, f1_score

y_pred = clf.predict(test_images)

y_pred_thresholded = (y_pred > 0.7).astype(int)

precision_metric = Precision()
precision_metric.update_state(test_labels, y_pred_thresholded)
precision = precision_metric.result().numpy()


recall_metric = Recall()
recall_metric.update_state(test_labels, y_pred_thresholded)
recall = recall_metric.result().numpy()

f1 = 2 * (precision * recall) / (precision + recall)

print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)

7/7 [==============================] - 28s 4s/step
Precision:  0.1919192
Recall:  0.11949685
F1 Score:  0.1472868315350143
Hamming Loss:  0.10101010101010101


In [ ]:
import h2o
from sklearn.preprocessing import MultiLabelBinarizer
from h2o.automl import H2OAutoML
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import img_to_array
from keras.models import Model
import numpy as np


In [ ]:
from sklearn.metrics import hamming_loss


test_predictions = model.predict(test_images)
test_predictions_binary = (test_predictions > 0.5).astype(int)


hamming_loss_value = hamming_loss(test_labels, test_predictions_binary)
print("Hamming Loss on Test Set:", hamming_loss_value)

7/7 [==============================] - 35s 4s/step
Hamming Loss on Test Set: 0.07300275482093664


In [ ]:
test_predictions

array([[0.07153588, 0.02793117, 0.05157837, ..., 0.05463211, 0.06178591,
        0.03282967],
       [0.04120564, 0.01876359, 0.075512  , ..., 0.033397  , 0.0761835 ,
        0.09263457],
       [0.01101265, 0.00878746, 0.12491807, ..., 0.0305884 , 0.05550271,
        0.1957823 ],
       ...,
       [0.01941689, 0.02331128, 0.06395928, ..., 0.0577538 , 0.05701064,
        0.09156447],
       [0.02722018, 0.0146514 , 0.08436105, ..., 0.0406079 , 0.0533265 ,
        0.13273093],
       [0.02074127, 0.00946162, 0.10720655, ..., 0.03719294, 0.05611887,
        0.1126641 ]], dtype=float32)